In [92]:
!pip install hnswlib polars  --quiet
!pip install shap gradio --quiet

In [93]:
#  Import libraries
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
import hnswlib
import polars as pl

import re
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb

from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import shap

# Suppress warnings
warnings.filterwarnings("ignore")

In [94]:
# Use alternative encoding to avoid UnicodeDecodeError
df = pl.read_csv('../data/fy2022_archived_opportunities.csv', encoding='latin-1')

# Get the first 5000 rows
#df = df.head(89000)
df

NoticeId,Title,Sol#,Department/Ind.Agency,CGAC,Sub-Tier,FPDS Code,Office,AAC Code,PostedDate,Type,BaseType,ArchiveType,ArchiveDate,SetASideCode,SetASide,ResponseDeadLine,NaicsCode,ClassificationCode,PopStreetAddress,PopCity,PopState,PopZip,PopCountry,Active,AwardNumber,AwardDate,Award$,Awardee,PrimaryContactTitle,PrimaryContactFullname,PrimaryContactEmail,PrimaryContactPhone,PrimaryContactFax,SecondaryContactTitle,SecondaryContactFullname,SecondaryContactEmail,SecondaryContactPhone,SecondaryContactFax,OrganizationType,State,City,ZipCode,CountryCode,AdditionalInfoLink,Link,Description
str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""97ec6ab3d5d4454192a8828827de6f…","""Whooster Web-Based Software""","""HQ003422P0171""","""DEPT OF DEFENSE""",97,"""WASHINGTON HEADQUARTERS SERVIC…","""97F5""","""WASHINGTON HEADQUARTERS SERVIC…","""HQ0034""","""2022-09-30 23:09:21.289-04""","""Award Notice""","""Special Notice""","""auto15""","""2022-10-14""","""""","""""","""""","""511210""","""7A21""","""""","""""","""""","""""","""""","""No""","""HQ003422P0171""","""2022-09-29""","""125000.00""","""WHOOSTER INC TX 78610-2850 USA""","""""","""Roya Sterner""","""roya.m.sterner.civ@mail.mil""","""5712254264""","""""","""""","""""","""""","""""","""""","""OFFICE""","""DC""","""WASHINGTON""","""203011000""","""USA""","""""","""https://sam.gov/opp/97ec6ab3d5…","""The Whooster Web Based Softwar…"
"""c15a7148317b44eeac210a7b640687…","""Beckman Coulter Reagents and S…","""W81K0222Q0162""","""DEPT OF DEFENSE""",21,"""DEPT OF THE ARMY""","""2100""","""W40M RHCO PACIFIC""","""W81K02""","""2022-09-30 23:01:45.502-04""","""Presolicitation""","""Sources Sought""","""auto15""","""""","""""","""""","""2022-10-11T10:00:00-10:00""","""325413""","""6550""","""""","""""","""""","""""","""USA""","""No""","""""","""""","""""","""""","""""","""Megumi Kuwada""","""megumi.kuwada.civ@mail.mil""","""8084385125""","""""","""""","""""","""""","""""","""""","""OFFICE""","""HI""","""FORT SHAFTER""","""96858-5098""","""USA""","""""","""https://sam.gov/opp/c15a714831…","""The Medical Readiness Contract…"
"""bf103afa7b4b455bb1541eb450e381…","""8(a) WebEOC IL5 Cloud Services…","""HQ003422C0130""","""DEPT OF DEFENSE""",97,"""WASHINGTON HEADQUARTERS SERVIC…","""97F5""","""WASHINGTON HEADQUARTERS SERVIC…","""HQ0034""","""2022-09-30 22:12:14.082-04""","""Award Notice""","""Award Notice""","""auto15""","""2022-10-14""","""8AN""","""8(a) Sole Source (FAR 19.8)""","""""","""541519""","""DF01""","""""","""""","""""","""""","""""","""No""","""HQ003422C0130""","""2022-09-29""","""4838341.62""","""SPAARK INC PA 17201 USA""","""""","""Roya Sterner""","""roya.m.sterner.civ@mail.mil""","""5712254264""","""""","""""","""""","""""","""""","""""","""OFFICE""","""DC""","""WASHINGTON""","""203011000""","""USA""","""""","""https://sam.gov/opp/bf103afa7b…","""Direct Award of an 8(a) contra…"
"""d1619b0084ac4d439585b727343b46…","""Leslie Valve IDIQ - Sources So…","""N4523A22RC04N81""","""DEPT OF DEFENSE""",17,"""DEPT OF THE NAVY""","""1700""","""PUGET SOUND NAVAL SHIPYARD IMF""","""N4523A""","""2022-09-30 22:05:45.599-04""","""Sources Sought""","""Sources Sought""","""auto15""","""""","""""","""""","""2022-10-14T11:00:00-07:00""","""811310""","""J048""","""""","""""","""""","""""","""USA""","""No""","""""","""""","""""","""""","""""","""Bart Franklin""","""bart.franklin@navy.mil""","""3609793786""","""""","""""","""Chris Parker""","""Christopher.L.Parke1@navy.mil""","""3609793943""","""""","""OFFICE""","""WA""","""BREMERTON""","""98314-5001""","""USA""","""""","""https://sam.gov/opp/d1619b0084…","""Puget Sound Naval Shipyard and…"
"""48b6b27846bd4155990cdc2f2be335…","""PFPA BALLISTIC VEST AND PLATES""","""HQ003422Q0206""","""DEPT OF DEFENSE""",97,"""WASHINGTON HEADQUARTERS SERVIC…","""97F5""","""WASHINGTON HEADQUARTERS SERVIC…","""HQ0034""","""2022-09-30 21:36:36.

In [95]:
# Function to clean column names
def clean_names(df):
    """
    Cleans column names of a Polars DataFrame by removing special characters,
    converting to lowercase, and replacing spaces with underscores.

    Args:
        df: Polars DataFrame

    Returns:
        Polars DataFrame with cleaned column names.
    """
    df.columns = [re.sub(r'\W+', '_', col).lower() for col in df.columns]
    return df

# Load your DataFrame
# df2023 = pl.read_csv('your_dataset.csv')  # Replace with your actual data loading method

# Clean column names
df2023 = clean_names(df)
df2023

noticeid,title,sol_,department_ind_agency,cgac,sub_tier,fpds_code,office,aac_code,posteddate,type,basetype,archivetype,archivedate,setasidecode,setaside,responsedeadline,naicscode,classificationcode,popstreetaddress,popcity,popstate,popzip,popcountry,active,awardnumber,awarddate,award_,awardee,primarycontacttitle,primarycontactfullname,primarycontactemail,primarycontactphone,primarycontactfax,secondarycontacttitle,secondarycontactfullname,secondarycontactemail,secondarycontactphone,secondarycontactfax,organizationtype,state,city,zipcode,countrycode,additionalinfolink,link,description
str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""97ec6ab3d5d4454192a8828827de6f…","""Whooster Web-Based Software""","""HQ003422P0171""","""DEPT OF DEFENSE""",97,"""WASHINGTON HEADQUARTERS SERVIC…","""97F5""","""WASHINGTON HEADQUARTERS SERVIC…","""HQ0034""","""2022-09-30 23:09:21.289-04""","""Award Notice""","""Special Notice""","""auto15""","""2022-10-14""","""""","""""","""""","""511210""","""7A21""","""""","""""","""""","""""","""""","""No""","""HQ003422P0171""","""2022-09-29""","""125000.00""","""WHOOSTER INC TX 78610-2850 USA""","""""","""Roya Sterner""","""roya.m.sterner.civ@mail.mil""","""5712254264""","""""","""""","""""","""""","""""","""""","""OFFICE""","""DC""","""WASHINGTON""","""203011000""","""USA""","""""","""https://sam.gov/opp/97ec6ab3d5…","""The Whooster Web Based Softwar…"
"""c15a7148317b44eeac210a7b640687…","""Beckman Coulter Reagents and S…","""W81K0222Q0162""","""DEPT OF DEFENSE""",21,"""DEPT OF THE ARMY""","""2100""","""W40M RHCO PACIFIC""","""W81K02""","""2022-09-30 23:01:45.502-04""","""Presolicitation""","""Sources Sought""","""auto15""","""""","""""","""""","""2022-10-11T10:00:00-10:00""","""325413""","""6550""","""""","""""","""""","""""","""USA""","""No""","""""","""""","""""","""""","""""","""Megumi Kuwada""","""megumi.kuwada.civ@mail.mil""","""8084385125""","""""","""""","""""","""""","""""","""""","""OFFICE""","""HI""","""FORT SHAFTER""","""96858-5098""","""USA""","""""","""https://sam.gov/opp/c15a714831…","""The Medical Readiness Contract…"
"""bf103afa7b4b455bb1541eb450e381…","""8(a) WebEOC IL5 Cloud Services…","""HQ003422C0130""","""DEPT OF DEFENSE""",97,"""WASHINGTON HEADQUARTERS SERVIC…","""97F5""","""WASHINGTON HEADQUARTERS SERVIC…","""HQ0034""","""2022-09-30 22:12:14.082-04""","""Award Notice""","""Award Notice""","""auto15""","""2022-10-14""","""8AN""","""8(a) Sole Source (FAR 19.8)""","""""","""541519""","""DF01""","""""","""""","""""","""""","""""","""No""","""HQ003422C0130""","""2022-09-29""","""4838341.62""","""SPAARK INC PA 17201 USA""","""""","""Roya Sterner""","""roya.m.sterner.civ@mail.mil""","""5712254264""","""""","""""","""""","""""","""""","""""","""OFFICE""","""DC""","""WASHINGTON""","""203011000""","""USA""","""""","""https://sam.gov/opp/bf103afa7b…","""Direct Award of an 8(a) contra…"
"""d1619b0084ac4d439585b727343b46…","""Leslie Valve IDIQ - Sources So…","""N4523A22RC04N81""","""DEPT OF DEFENSE""",17,"""DEPT OF THE NAVY""","""1700""","""PUGET SOUND NAVAL SHIPYARD IMF""","""N4523A""","""2022-09-30 22:05:45.599-04""","""Sources Sought""","""Sources Sought""","""auto15""","""""","""""","""""","""2022-10-14T11:00:00-07:00""","""811310""","""J048""","""""","""""","""""","""""","""USA""","""No""","""""","""""","""""","""""","""""","""Bart Franklin""","""bart.franklin@navy.mil""","""3609793786""","""""","""""","""Chris Parker""","""Christopher.L.Parke1@navy.mil""","""3609793943""","""""","""OFFICE""","""WA""","""BREMERTON""","""98314-5001""","""USA""","""""","""https://sam.gov/opp/d1619b0084…","""Puget Sound Naval Shipyard and…"
"""48b6b27846bd4155990cdc2f2be335…","""PFPA BALLISTIC VEST AND PLATES""","""HQ003422Q0206""","""DEPT OF DEFENSE""",97,"""WASHINGTON HEADQUARTERS SERVIC…","""97F5""","""WASHINGTON HEADQUARTERS SERVIC…","""HQ0034""","""2022-09-30 21:36:36.

In [96]:
# List of categorical columns
categorical_cols = ['title', 'sol_', 'department_ind_agency', 'sub_tier', 'fpds_code', 'office', 'aac_code', 
                    'posteddate', 'type', 'basetype', 'archivetype', 'archivedate', 'setasidecode', 'setaside', 
                    'responsedeadline', 'naicscode', 'classificationcode', 'popstreetaddress', 'popcity', 
                    'popstate', 'popzip', 'popcountry', 'active', 'awardnumber', 'awarddate', 'awardee', 
                    'primarycontacttitle', 'primarycontactfullname', 'primarycontactemail', 'primarycontactphone', 
                    'primarycontactfax', 'secondarycontacttitle', 'secondarycontactfullname', 'secondarycontactemail', 
                    'secondarycontactphone', 'secondarycontactfax', 'organizationtype', 'state', 'city', 'zipcode', 
                    'countrycode', 'additionalinfolink', 'link', 'description']

# Convert all categorical columns to strings
df2023 = df2023.with_columns([pl.col(col).cast(pl.Utf8) for col in categorical_cols])
df2023

noticeid,title,sol_,department_ind_agency,cgac,sub_tier,fpds_code,office,aac_code,posteddate,type,basetype,archivetype,archivedate,setasidecode,setaside,responsedeadline,naicscode,classificationcode,popstreetaddress,popcity,popstate,popzip,popcountry,active,awardnumber,awarddate,award_,awardee,primarycontacttitle,primarycontactfullname,primarycontactemail,primarycontactphone,primarycontactfax,secondarycontacttitle,secondarycontactfullname,secondarycontactemail,secondarycontactphone,secondarycontactfax,organizationtype,state,city,zipcode,countrycode,additionalinfolink,link,description
str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""97ec6ab3d5d4454192a8828827de6f…","""Whooster Web-Based Software""","""HQ003422P0171""","""DEPT OF DEFENSE""",97,"""WASHINGTON HEADQUARTERS SERVIC…","""97F5""","""WASHINGTON HEADQUARTERS SERVIC…","""HQ0034""","""2022-09-30 23:09:21.289-04""","""Award Notice""","""Special Notice""","""auto15""","""2022-10-14""","""""","""""","""""","""511210""","""7A21""","""""","""""","""""","""""","""""","""No""","""HQ003422P0171""","""2022-09-29""","""125000.00""","""WHOOSTER INC TX 78610-2850 USA""","""""","""Roya Sterner""","""roya.m.sterner.civ@mail.mil""","""5712254264""","""""","""""","""""","""""","""""","""""","""OFFICE""","""DC""","""WASHINGTON""","""203011000""","""USA""","""""","""https://sam.gov/opp/97ec6ab3d5…","""The Whooster Web Based Softwar…"
"""c15a7148317b44eeac210a7b640687…","""Beckman Coulter Reagents and S…","""W81K0222Q0162""","""DEPT OF DEFENSE""",21,"""DEPT OF THE ARMY""","""2100""","""W40M RHCO PACIFIC""","""W81K02""","""2022-09-30 23:01:45.502-04""","""Presolicitation""","""Sources Sought""","""auto15""","""""","""""","""""","""2022-10-11T10:00:00-10:00""","""325413""","""6550""","""""","""""","""""","""""","""USA""","""No""","""""","""""","""""","""""","""""","""Megumi Kuwada""","""megumi.kuwada.civ@mail.mil""","""8084385125""","""""","""""","""""","""""","""""","""""","""OFFICE""","""HI""","""FORT SHAFTER""","""96858-5098""","""USA""","""""","""https://sam.gov/opp/c15a714831…","""The Medical Readiness Contract…"
"""bf103afa7b4b455bb1541eb450e381…","""8(a) WebEOC IL5 Cloud Services…","""HQ003422C0130""","""DEPT OF DEFENSE""",97,"""WASHINGTON HEADQUARTERS SERVIC…","""97F5""","""WASHINGTON HEADQUARTERS SERVIC…","""HQ0034""","""2022-09-30 22:12:14.082-04""","""Award Notice""","""Award Notice""","""auto15""","""2022-10-14""","""8AN""","""8(a) Sole Source (FAR 19.8)""","""""","""541519""","""DF01""","""""","""""","""""","""""","""""","""No""","""HQ003422C0130""","""2022-09-29""","""4838341.62""","""SPAARK INC PA 17201 USA""","""""","""Roya Sterner""","""roya.m.sterner.civ@mail.mil""","""5712254264""","""""","""""","""""","""""","""""","""""","""OFFICE""","""DC""","""WASHINGTON""","""203011000""","""USA""","""""","""https://sam.gov/opp/bf103afa7b…","""Direct Award of an 8(a) contra…"
"""d1619b0084ac4d439585b727343b46…","""Leslie Valve IDIQ - Sources So…","""N4523A22RC04N81""","""DEPT OF DEFENSE""",17,"""DEPT OF THE NAVY""","""1700""","""PUGET SOUND NAVAL SHIPYARD IMF""","""N4523A""","""2022-09-30 22:05:45.599-04""","""Sources Sought""","""Sources Sought""","""auto15""","""""","""""","""""","""2022-10-14T11:00:00-07:00""","""811310""","""J048""","""""","""""","""""","""""","""USA""","""No""","""""","""""","""""","""""","""""","""Bart Franklin""","""bart.franklin@navy.mil""","""3609793786""","""""","""""","""Chris Parker""","""Christopher.L.Parke1@navy.mil""","""3609793943""","""""","""OFFICE""","""WA""","""BREMERTON""","""98314-5001""","""USA""","""""","""https://sam.gov/opp/d1619b0084…","""Puget Sound Naval Shipyard and…"
"""48b6b27846bd4155990cdc2f2be335…","""PFPA BALLISTIC VEST AND PLATES""","""HQ003422Q0206""","""DEPT OF DEFENSE""",97,"""WASHINGTON HEADQUARTERS SERVIC…","""97F5""","""WASHINGTON HEADQUARTERS SERVIC…","""HQ0034""","""2022-09-30 21:36:36.

In [ ]:
print(df2023)

shape: (323_030, 47)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ noticeid  ┆ title     ┆ sol_      ┆ departmen ┆ … ┆ countryco ┆ additiona ┆ link      ┆ descript │
│ ---       ┆ ---       ┆ ---       ┆ t_ind_age ┆   ┆ de        ┆ linfolink ┆ ---       ┆ ion      │
│ str       ┆ str       ┆ str       ┆ ncy       ┆   ┆ ---       ┆ ---       ┆ str       ┆ ---      │
│           ┆           ┆           ┆ ---       ┆   ┆ str       ┆ str       ┆           ┆ str      │
│           ┆           ┆           ┆ str       ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 97ec6ab3d ┆ Whooster  ┆ HQ003422P ┆ DEPT OF   ┆ … ┆ USA       ┆           ┆ https://s ┆ The      │
│ 5d4454192 ┆ Web-Based ┆ 0171      ┆ DEFENSE   ┆   ┆           ┆           ┆ am.gov/op ┆ Whooster │
│ a8828827d ┆ Software  ┆           ┆           ┆   ┆           ┆     

: 

In [ ]:
# Apply LabelEncoder to each categorical column
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df2023 = df2023.with_columns([pl.Series(col, le.fit_transform(df2023[col].to_list()))])
    label_encoders[col] = le

# Ensure numerical columns are in the correct format

df2023

In [ ]:
# Separate numeric and non-numeric columns
numeric_cols = df2023.select(pl.col(pl.Float64) | pl.col(pl.Int64)).columns
non_numeric_cols = df2023.select(pl.col(pl.Utf8)).columns
non_numeric_cols

In [ ]:
# Impute missing values for numeric columns
imputer = SimpleImputer(strategy='mean')
imputer

In [ ]:
df2023

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df2023_df):
    # Filter rows based on column: 'award_'
    df2023_df = df2023_df[(df2023_df['award_'].notna()) & (df2023_df['award_'] != "")]
    return df2023_df

# Loaded variable 'df2023' from kernel state
df2023_df = df2023.to_pandas()

df2023_df_clean = clean_data(df2023_df.copy())
df2023_df_clean.head()

In [ ]:
df2023_df_clean.columns

In [ ]:
# Define features and target
X = df2023_df_clean.drop('award_', axis=1)
y = df2023_df_clean['award_']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
print(y_train)

In [ ]:
print(y_test)

In [ ]:
# Function to clean currency columns
def clean_currency_columns(df):
    currency_cols = df.columns[df.apply(lambda col: col.astype(str).str.contains(r'\$').any())]
    for col in currency_cols:
        df[col] = df[col].str.replace('[\$,]', '', regex=True).astype(float)
    return df

# Clean X_train and X_test
X_train = clean_currency_columns(X_train)
X_test = clean_currency_columns(X_test)


In [ ]:
X_train

In [ ]:
# Train XGBoost model with enable_categorical=True
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    random_state=42,
    enable_categorical=True
)

In [ ]:
X_train.columns

In [ ]:
# Find columns starting with 'aw'
a_columns = [col for col in X_train.columns if col.startswith('a')]
print("Columns starting with 'aw':", a_columns)

# Display the first few rows of X_train
print(X_train[a_columns])

X_train=X_train[a_columns]

In [ ]:
y_train.head()

In [ ]:
# Remove '$' and ',' then convert to float
y_train = y_train.str.replace(r'[\$,]', '', regex=True).astype(float)
y_test = y_test.str.replace(r'[\$,]', '', regex=True).astype(float)

In [ ]:
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)],
    verbose=True
)

In [ ]:
# Check the number of columns
print("Number of columns in X_train:", X_train.shape[1])
print("Number of columns in X_test:", X_test.shape[1])

# Check for column differences
diff_train_test = set(X_train.columns) - set(X_test.columns)
diff_test_train = set(X_test.columns) - set(X_train.columns)

print("Columns in X_train not in X_test:", diff_train_test)
print("Columns in X_test not in X_train:", diff_test_train)

# Align X_test columns to X_train
X_test = X_test[X_train.columns]

# Fit the model
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)],
    verbose=True
)

In [ ]:
xgb_predictions = xgb_model.predict(X_test)

# Evaluate the model
xgb_mse = mean_squared_error(y_test, xgb_predictions)
xgb_r2 = r2_score(y_test, xgb_predictions)

print(f"XGBoost MSE: {xgb_mse}, R2: {xgb_r2}")

# Plot the results
plt.figure(figsize=(7, 6))
plt.scatter(y_test, xgb_predictions, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('XGBoost Predictions')
plt.show()

In [ ]:
import pandas as pd

def clean_column_names(df):
    # Replace spaces and slashes with underscores
    df.columns = [col.replace(' ', '_').replace('/', '_') for col in df.columns]
    # Convert to lowercase
    df.columns = [col.lower() for col in df.columns]
    return df

# Assuming df2023 is your DataFrame
df2023 = clean_column_names(df)
df2023.columns

In [ ]:
# Define prediction function to match provided columns
def predict_popularity(title, department_agency, cgac, sub_tier, fpds_code, office, aac_code, posted_date, contract_type, base_type, archive_type, archive_date, set_aside_code, set_aside, response_deadline, naics_code, classification_code, pop_street_address, pop_city, pop_state, pop_zip, pop_country, active, award_number, award_date, award_amount, awardee, primary_contact_title, primary_contact_fullname, primary_contact_email, primary_contact_phone, primary_contact_fax, secondary_contact_title, secondary_contact_fullname, secondary_contact_email, secondary_contact_phone, secondary_contact_fax, organization_type, state, city, zip_code, country_code, additional_info_link, link, description):

    # Hash specific text fields to match the model's expected input format
    title_hashed = hash(str(title)) % 10**8
    department_agency_hashed = hash(str(department_agency)) % 10**8
    sub_tier_hashed = hash(str(sub_tier)) % 10**8
    awardee_hashed = hash(str(awardee)) % 10**8

    # Create a DataFrame for the input, matching the column names in X
    input_data = pd.DataFrame([[
        title_hashed, department_agency_hashed, cgac, sub_tier_hashed, fpds_code,
        office, aac_code, posted_date, contract_type, base_type, archive_type, archive_date,
        set_aside_code, set_aside, response_deadline, naics_code, classification_code,
        pop_street_address, pop_city, pop_state, pop_zip, pop_country, active,
        award_number, award_date, award_amount, awardee_hashed, primary_contact_title,
        primary_contact_fullname, primary_contact_email, primary_contact_phone,
        primary_contact_fax, secondary_contact_title, secondary_contact_fullname,
        secondary_contact_email, secondary_contact_phone, secondary_contact_fax,
        organization_type, state, city, zip_code, country_code, additional_info_link,
        link, description
    ]], columns=X.columns)

    # Define numeric columns, if any, to normalize
    # (Assume 'award_amount' as a numerical example, modify as needed based on your data)
    numerical_cols = ['award_amount']
    input_data[numerical_cols] = (input_data[numerical_cols] - X[numerical_cols].mean()) / X[numerical_cols].std()

    # Return the DataFrame ready for prediction
    return input_data


In [ ]:
X_train.dtypes.to_frame('Data Type')

In [ ]:
# Missing values in each row
missing_values_per_row = X_train.isnull().sum(axis=1)
count_per_missing_value = missing_values_per_row.value_counts().sort_index()

# Print the results
for missing, rows in count_per_missing_value.items():
    print(f'{rows} row(s) have {missing} missing values')

total_rows_with_missing_values = (X_train.isnull().any(axis=1)).sum()
print(f'Total number of rows with missing values: {total_rows_with_missing_values}')

In [ ]:
dist_numerical_cols = X_train.describe().T[['min', 'max']]
dist_numerical_cols['Missing Values'] = X_train.isnull().sum()
dist_numerical_cols['Missing Percentage'] = (X_train.isnull().mean() * 100).round(2)

The output shows that all numeric columns have a missing value percentage of 0, indicating that **there are no missing values** in these columns.

To gain further insights into the data, we used the `describe()` method to generate a comprehensive statistical summary of the numerical columns.

**👉 Distribution of numerical columns**

In [ ]:
sns.set_style('darkgrid')
sns.set(rc={"axes.facecolor":"#F2EAC5","figure.facecolor":"#F2EAC5"})
X_train.hist(figsize=(20,15), bins=30, xlabelsize=8, ylabelsize=8)
plt.tight_layout()
plt.show()

In [ ]:
X_train.info()

In [ ]:
# Import necessary libraries
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Fit Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

# Fit Ridge Regression model
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)
ridge_predictions = ridge_model.predict(X_test)

# Evaluate the models
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)

ridge_mse = mean_squared_error(y_test, ridge_predictions)
ridge_r2 = r2_score(y_test, ridge_predictions)

print(f"Random Forest MSE: {rf_mse}, R2: {rf_r2}")
print(f"Ridge Regression MSE: {ridge_mse}, R2: {ridge_r2}")


In [ ]:
# Plot the results
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.scatter(y_test, rf_predictions, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Random Forest Predictions')

plt.subplot(1, 2, 2)
plt.scatter(y_test, ridge_predictions, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Ridge Regression Predictions')

plt.tight_layout()
plt.show()

🖥️ We use `matplotlib_venn` to plot the venn diagrams so we need to install it first (if not already installed).